# Initialization

In [2]:
%load_ext autoreload
%load_ext line_profiler
import sys
sys.path.append("..")

In [129]:
%autoreload

import numpy as np
import pickle

import main


import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc_file('matplotlibrc')

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.spectools as spectools

from darkhistory.spec.spectrum import Spectrum

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.elec_cooling import get_elec_cooling_tf_fast

from darkhistory.electrons import positronium

from scipy.interpolate import interp1d


In [4]:
ics_thomson_ref_tf=pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/ics_thomson_ref_tf.raw", "rb")) 
ics_rel_ref_tf=pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/ics_rel_ref_tf.raw", "rb"))
engloss_ref_tf=pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/engloss_ref_tf.raw", "rb"))

In [9]:
%autoreload

eleceng = 10**np.arange(0, np.log10(5e12), 0.0254)
photeng = 10**np.arange(-4, np.log10(5e12), 0.0334)


rs = 600
xe = 1e-2

from darkhistory.spec.spectra import Spectra

coll_ion_sec_elec_specs = (
    phys.coll_ion_sec_elec_spec(eleceng, eleceng, species='HI'),
    phys.coll_ion_sec_elec_spec(eleceng, eleceng, species='HeI'),
    phys.coll_ion_sec_elec_spec(eleceng, eleceng, species='HeII')
)

coll_exc_sec_elec_tf_HI = tf.TransFuncAtRedshift(
    np.squeeze(np.identity(eleceng.size)[:, np.where(eleceng > phys.lya_eng)]),
    in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    eng = eleceng[eleceng > phys.lya_eng] - phys.lya_eng,
    dlnz = -1, spec_type = 'N'
)

coll_exc_sec_elec_tf_HeI = tf.TransFuncAtRedshift(
    np.squeeze(np.identity(eleceng.size)[:, np.where(eleceng > phys.He_exc_eng)]),
    in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    eng = eleceng[eleceng > phys.He_exc_eng] - phys.He_exc_eng,
    dlnz = -1, spec_type = 'N'
)

coll_exc_sec_elec_tf_HeII = tf.TransFuncAtRedshift(
    np.squeeze(np.identity(eleceng.size)[:, np.where(eleceng > 4*phys.lya_eng)]),
    in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    eng = eleceng[eleceng > 4*phys.lya_eng] - 4*phys.lya_eng,
    dlnz = -1, spec_type = 'N'
)

coll_exc_sec_elec_tf_HI.rebin(eleceng)
coll_exc_sec_elec_tf_HeI.rebin(eleceng)
coll_exc_sec_elec_tf_HeII.rebin(eleceng)

coll_exc_sec_elec_specs = (
    coll_exc_sec_elec_tf_HI.grid_vals,
    coll_exc_sec_elec_tf_HeI.grid_vals,
    coll_exc_sec_elec_tf_HeII.grid_vals
)

from darkhistory.spec.spectools import EnglossRebinData

ics_engloss_data = EnglossRebinData(eleceng, photeng, eleceng)


../darkhistory/spec/spectra.py:401: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  np.issubdtype(type(other), float)
../darkhistory/spec/spectra.py:402: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  or np.issubdtype(type(other), int)


In [130]:
%autoreload

get_elec_cooling_tf_fast(
    ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf, 
    coll_ion_sec_elec_specs, coll_exc_sec_elec_specs, 
    eleceng, photeng, rs, xe, xHe=0, ics_engloss_data = ics_engloss_data, check_conservation_eng=True, verbose=False
)

(<darkhistory.spec.transferfunction.TransFuncAtRedshift at 0x10e5b96a0>,
 array([   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,   -0.        ,
          -0.        ,   -0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,   -0.        ,   -0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.

# With Linear Algebra

In [118]:
def get_elec_cooling_tf_fast_linalg(
    raw_nonrel_tf, raw_rel_tf, raw_engloss_tf,
    coll_ion_sec_elec_specs, coll_exc_sec_elec_specs,
    eleceng, photeng, rs, xe, xHe=0, ics_engloss_data=None, 
    check_conservation_eng = False, verbose=False
):

    """Returns transfer function for complete electron cooling through ICS and atomic processes.

    Parameters
    ----------
    nonrel_tf : TransFuncAtRedshift
        Raw nonrelativistic primary electron ICS transfer function.
    rel_tf : string
        Raw relativistic primary electron ICS transfer function.
    engloss_tf_filename : string
        Raw primary electron ICS energy loss transfer function.
    coll_ion_sec_elec_specs : tuple of ndarray
        Normalized collisional ionization secondary electron spectra, order HI, HeI, HeII, indexed by eleceng (injection) x eleceng (abscissa).
    coll_exc_sec_elec_specs : tuple of ndarray
        Normalized collisional excitation secondary electron spectra, order HI, HeI, HeII, indexed by eleceng (injection) x eleceng (abscissa).
    eleceng : ndarray
        The electron *kinetic* energy abscissa.
    photeng : ndarray
        The photon energy abscissa.
    rs : float
        The redshift.
    xe : float
        Free electron fraction. 
    xHe : float, optional
        Singly-ionized helium fraction, nHe+/nH. Set to nHe/nH*xe if None.
    ics_engloss_data : EnglossRebinData
        Stores rebinning information for speed. 
    check_conservation_eng : bool
        If true, checks for energy conservation.
    verbose : bool
        If true, prints energy conservation checks.
    
    Returns
    -------

    tuple of TransFuncAtRedshift
        Transfer functions for photons and low energy electrons.

    Note
    ----
    The raw transfer functions should be generated when the code package is first installed. The transfer function corresponds to the fully resolved
    photon spectrum after scattering by one electron.

    This version of the code works faster, but dispenses with energy conservation checks and several other safeguards. Use only with default abscissa, or when get_ics_cooling_tf works.

    """

    if xHe is None:
        xHe = xe*phys.nHe/phys.nH
        
    # v/c of electrons, important subsequently.
    beta_ele = np.sqrt(1 - 1/(1 + eleceng/phys.me)**2)
        
    #####################################
    # Inverse Compton
    #####################################

    T = phys.TCMB(rs)

    # Photon transfer function for single primary electron single scattering.
    # This is dN/(dE dt), dt = 1 s.
    phot_ICS_tf = ics_spec(
        eleceng, photeng, T, nonrel_tf = raw_nonrel_tf, rel_tf = raw_rel_tf
    )

    # Downcasting speeds up np.dot
    phot_ICS_tf._grid_vals = phot_ICS_tf.grid_vals.astype('float64')

    # Energy loss transfer function for single primary electron
    # single scattering. This is dN/(dE dt), dt = 1 s.
    engloss_ICS_tf = engloss_spec(
        eleceng, photeng, T, nonrel_tf = raw_engloss_tf, rel_tf = raw_rel_tf
    )

    # Downcasting speeds up np.dot
    engloss_ICS_tf._grid_vals = engloss_ICS_tf.grid_vals.astype('float64')

    # Switch the spectra type here to type 'N'.
    if phot_ICS_tf.spec_type == 'dNdE':
        phot_ICS_tf.switch_spec_type()
    if engloss_ICS_tf.spec_type == 'dNdE':
        engloss_ICS_tf.switch_spec_type()


    # Define some useful lengths.
    N_eleceng = eleceng.size
    N_photeng = photeng.size

    # Create the secondary electron transfer functions.

    # ICS transfer function.
    elec_ICS_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_eleceng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = eleceng,
        dlnz = -1, spec_type = 'N'
    )

    if ics_engloss_data is not None:
        elec_ICS_tf._grid_vals = ics_engloss_data.rebin(
            engloss_ICS_tf.grid_vals
        )
    else:
        elec_ICS_tf._grid_vals = spectools.engloss_rebin_fast(
            eleceng, photeng, engloss_ICS_tf.grid_vals, eleceng
        )
    
    # Total upscattered photon energy.
    cont_loss_ICS_vec = np.zeros_like(eleceng)
    # Deposited energy, enforces energy conservation.
    deposited_ICS_vec = np.zeros_like(eleceng)
    
    
    #####################
    # Excitation  
    #####################
    
    # Construct the rate matrices first. Secondary electron spectrum is an electron at in_eng - excitation energy, 
    # with a per second rate given by n*sigma*c.
    

    # rate_matrix_exc_HI = np.diag(
    #     (1 - xe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HI') * beta_ele * phys.c
    # )
    
    # rate_matrix_exc_HeI = np.diag(
    #     (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeI') * beta_ele * phys.c
    # )
    
    # rate_matrix_exc_HeII = np.diag(
    #     xHe*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeII') * beta_ele * phys.c
    # )

    # Construct the TransFuncAtRedshift objects.
    # Electrons scatter from in_eng to in_eng - excitation energy.
    # Remove all of the columns (eng) that have energies below the excitation energy, 
    # elec_exc_HI_tf = tf.TransFuncAtRedshift(
    #     np.squeeze(rate_matrix_exc_HI[:, np.where(eleceng > phys.lya_eng)]), 
    #     in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    #     eng = eleceng[eleceng > phys.lya_eng] - phys.lya_eng,
    #     dlnz = -1, spec_type = 'N'
    # )
    # elec_exc_HeI_tf = tf.TransFuncAtRedshift(
    #     np.squeeze(rate_matrix_exc_HeI[:, np.where(eleceng > phys.He_exc_eng)]), 
    #     in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    #     eng = eleceng[eleceng > phys.He_exc_eng] - phys.He_exc_eng,
    #     dlnz = -1, spec_type = 'N'
    # )
    # elec_exc_HeII_tf = tf.TransFuncAtRedshift(
    #     np.squeeze(rate_matrix_exc_HeII[:, np.where(eleceng > 4*phys.lya_eng)]), 
    #     in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
    #     eng = eleceng[eleceng > 4*phys.lya_eng] - 4*phys.lya_eng,
    #     dlnz = -1, spec_type = 'N'
    # )
    
    # Rebin these transfer functions back to eleceng.
    # elec_exc_HI_tf.rebin(eleceng)
    # elec_exc_HeI_tf.rebin(eleceng)
    # elec_exc_HeII_tf.rebin(eleceng)

    rate_vec_exc_HI = (
        (1 - xe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HI') * beta_ele * phys.c
    )
    
    rate_vec_exc_HeI = (
        (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeI') * beta_ele * phys.c
    )
    
    rate_vec_exc_HeII = (
        xHe*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeII') * beta_ele * phys.c
    )

    elec_exc_HI_tf = tf.TransFuncAtRedshift(
        rate_vec_exc_HI[:, np.newaxis]*coll_exc_sec_elec_specs[0],
        in_eng = eleceng, rs = rs*np.ones_like(eleceng),
        eng = eleceng, dlnz = -1, spec_type  = 'N'
    )

    elec_exc_HeI_tf = tf.TransFuncAtRedshift(
        rate_vec_exc_HeI[:, np.newaxis]*coll_exc_sec_elec_specs[1],
        in_eng = eleceng, rs = rs*np.ones_like(eleceng),
        eng = eleceng, dlnz = -1, spec_type  = 'N'
    )

    elec_exc_HeII_tf = tf.TransFuncAtRedshift(
        rate_vec_exc_HeII[:, np.newaxis]*coll_exc_sec_elec_specs[2],
        in_eng = eleceng, rs = rs*np.ones_like(eleceng),
        eng = eleceng, dlnz = -1, spec_type  = 'N'
    )
   
    # Deposited energy for excitation.
    deposited_exc_vec = np.zeros_like(eleceng)
    
    #####################
    # Ionization  
    #####################
    
    # Construct the rate vector first. Secondary electron spectrum is an electron at in_eng - excitation energy, 
    # with a per second rate given by n*sigma*c.
    rate_vec_ion_HI = (
        (1 - xe)*phys.nH*rs**3 
        * phys.coll_ion_xsec(eleceng, species='HI') * beta_ele * phys.c
    )
    
    rate_vec_ion_HeI = (
        (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 
        * phys.coll_ion_xsec(eleceng, species='HeI') * beta_ele * phys.c
    )
    
    rate_vec_ion_HeII = (
        xHe*phys.nH*rs**3 * 
        phys.coll_ion_xsec(eleceng, species='HeII') * beta_ele * phys.c
    )
    
    # Construct the spectra. 
    # elec_spec_ion_HI = np.array(
    #     [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HI') 
    # for rate,in_eng in zip(rate_vec_ion_HI,eleceng)]
    # )
    # elec_spec_ion_HeI = np.array(
    #     [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HeI') 
    # for rate,in_eng in zip(rate_vec_ion_HeI,eleceng)]
    # )
    # elec_spec_ion_HeII = np.array(
    #     [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HeII') 
    # for rate,in_eng in zip(rate_vec_ion_HeII,eleceng)]
    # )

    # transpose to force multiplication along first axis.
    elec_spec_ion_HI   = (
        rate_vec_ion_HI[:,np.newaxis]   * coll_ion_sec_elec_specs[0]
    )
    elec_spec_ion_HeI  = (
        rate_vec_ion_HeI[:,np.newaxis]  * coll_ion_sec_elec_specs[1]
    )
    elec_spec_ion_HeII = (
        rate_vec_ion_HeII[:,np.newaxis] * coll_ion_sec_elec_specs[2]
    )
    
    # Construct the TransFuncAtRedshift objects.
    # Electrons scatter from in_eng to in_eng - excitation energy.
    # Remove all of the columns (eng) that have energies below the 
    # excitation energy, 
    elec_ion_HI_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HI, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    elec_ion_HeI_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HeI, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    elec_ion_HeII_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HeII, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    
    # Deposited energy for ionization.
    deposited_ion_vec = np.zeros_like(eleceng)
    
    #############################################
    # Heating
    #############################################
    
    dE_heat_dt = phys.elec_heating_engloss_rate(eleceng, xe, rs)
    
    deposited_heat_vec = np.zeros_like(eleceng)

    # new_eleceng = eleceng - dE_heat_dt

    # if not np.all(new_eleceng[1:] > eleceng[:-1]):
    #     utils.compare_arr([new_eleceng, eleceng])
    #     raise ValueError('heating loss is too large: smaller time step required.')

    # # After the check above, we can define the spectra by
    # # manually assigning slightly less than 1 particle along
    # # diagonal, and a small amount in the bin below. 

    # # N_n-1 E_n-1 + N_n E_n = E_n - dE_dt
    # # N_n-1 + N_n = 1
    # # therefore, (1 - N_n) E_n-1 - (1 - N_n) E_n = - dE_dt
    # # i.e. N_n = 1 + dE_dt/(E_n-1 - E_n)

    elec_heat_spec_grid = np.identity(eleceng.size)
    elec_heat_spec_grid[0,0] -= dE_heat_dt[0]/eleceng[0]
    elec_heat_spec_grid[1:, 1:] += np.diag(
        dE_heat_dt[1:]/(eleceng[:-1] - eleceng[1:])
    )
    elec_heat_spec_grid[1:, :-1] -= np.diag(
        dE_heat_dt[1:]/(eleceng[:-1] - eleceng[1:])
    )

    
    
    #############################################
    # Initialization of secondary spectra 
    #############################################

    # Low and high energy boundaries
    loweng = 3000
    eleceng_high = eleceng[eleceng > loweng]
    eleceng_high_ind = np.arange(eleceng.size)[eleceng > loweng]
    eleceng_low = eleceng[eleceng <= loweng]
    eleceng_low_ind  = np.arange(eleceng.size)[eleceng <= loweng]


    if eleceng_low.size == 0:
        raise TypeError('Energy abscissa must contain a low energy bin below 3 keV.')

    # Empty containers for quantities.
    # Final secondary photon spectrum.
    sec_phot_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_photeng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = photeng,
        dlnz = -1, spec_type = 'N'
    )
    # Final secondary low energy electron spectrum.
    sec_lowengelec_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_eleceng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = eleceng,
        dlnz = -1, spec_type = 'N'
    )

    # Start building sec_phot_tf and sec_lowengelec_tf.
    # Low energy regime first.

    sec_lowengelec_tf._grid_vals[:eleceng_low.size, :eleceng_low.size] = (
        np.identity(eleceng_low.size)
    )

    # Continuum energy loss rate per electron, dU_CMB/dt.
    CMB_upscatter_eng_rate = phys.thomson_xsec*phys.c*phys.CMB_eng_density(T)
    
    sec_elec_spec_N_arr = (
        elec_ICS_tf.grid_vals
        + elec_exc_HI_tf.grid_vals + elec_exc_HeI_tf.grid_vals + elec_exc_HeII_tf.grid_vals
        + elec_ion_HI_tf.grid_vals + elec_ion_HeI_tf.grid_vals + elec_ion_HeII_tf.grid_vals
        + elec_heat_spec_grid
    )
    
    sec_phot_spec_N_arr = phot_ICS_tf.grid_vals
    
    sec_elec_totN_arr = np.sum(sec_elec_spec_N_arr, axis=1)
    sec_elec_toteng_arr = np.dot(sec_elec_spec_N_arr, eleceng)
    sec_phot_toteng_arr = np.dot(sec_phot_spec_N_arr, photeng)
    
    deposited_ICS_eng_arr = (
        np.sum(elec_ICS_tf.grid_vals, axis=1)*eleceng
        - np.dot(elec_ICS_tf.grid_vals, eleceng)
        - (np.dot(sec_phot_spec_N_arr, photeng) - CMB_upscatter_eng_rate)
    )
    
    deposited_ICS_eng_arr[eleceng > 20*phys.me - phys.me] -= CMB_upscatter_eng_rate
    continuum_engloss_arr = CMB_upscatter_eng_rate*np.ones_like(eleceng)
    continuum_engloss_arr[eleceng > 20*phys.me - phys.me] = 0
    
    
    deposited_exc_eng_arr = (
        phys.lya_eng*np.sum(elec_exc_HI_tf.grid_vals, axis=1)
        + phys.He_exc_eng*np.sum(elec_exc_HeI_tf.grid_vals, axis=1)
        + 4*phys.lya_eng*np.sum(elec_exc_HeII_tf.grid_vals, axis=1)
    )
    
    deposited_ion_eng_arr = (
        phys.rydberg*np.sum(elec_ion_HI_tf.grid_vals, axis=1)/2
        + phys.He_ion_eng*np.sum(elec_ion_HeI_tf.grid_vals, axis=1)/2
        + 4*phys.rydberg*np.sum(elec_ion_HeII_tf.grid_vals, axis=1)/2
    )
    deposited_heat_eng_arr = dE_heat_dt
    
    # remove self-scattering, re-normalize. 
    np.fill_diagonal(sec_elec_spec_N_arr, 0)
    
    toteng_no_self_scatter_arr = (
        np.dot(sec_elec_spec_N_arr, eleceng)
        + np.dot(sec_phot_spec_N_arr, photeng)
        - continuum_engloss_arr
        + deposited_ICS_eng_arr
        + deposited_exc_eng_arr
        + deposited_ion_eng_arr
        + deposited_heat_eng_arr
    )
    
    fac_arr = eleceng/toteng_no_self_scatter_arr
    
    sec_elec_spec_N_arr *= fac_arr[:, np.newaxis]
    sec_phot_spec_N_arr *= fac_arr[:, np.newaxis]
    continuum_engloss_arr  *= fac_arr
    deposited_ICS_eng_arr  *= fac_arr
    deposited_exc_eng_arr  *= fac_arr
    deposited_ion_eng_arr  *= fac_arr
    deposited_heat_eng_arr *= fac_arr
    
    deposited_ICS_eng_arr[eleceng < loweng]  = 0
    deposited_exc_eng_arr[eleceng < loweng]  = 0
    deposited_ion_eng_arr[eleceng < loweng]  = 0
    deposited_heat_eng_arr[eleceng < loweng] = 0
    
    continuum_engloss_arr[eleceng < loweng]  = 0
    
    sec_phot_spec_N_arr[eleceng < loweng] = 0
    
    sec_lowengelec_N_arr = np.identity(eleceng.size)
    sec_lowengelec_N_arr[eleceng >= loweng] = 0
    sec_lowengelec_N_arr[eleceng_high_ind[0]:, :eleceng_high_ind[0]] += sec_elec_spec_N_arr[eleceng_high_ind[0]:, :eleceng_high_ind[0]]

    sec_highengelec_N_arr = np.zeros_like(sec_elec_spec_N_arr)
    sec_highengelec_N_arr[:, eleceng_high_ind[0]:] = sec_elec_spec_N_arr[:, eleceng_high_ind[0]:]
    
    # T = E.T + Prompt
    deposited_ICS_vec_linalg  = np.linalg.solve(np.identity(eleceng.size) - sec_elec_spec_N_arr, deposited_ICS_eng_arr)
    deposited_exc_vec_linalg  = np.linalg.solve(np.identity(eleceng.size) - sec_elec_spec_N_arr, deposited_exc_eng_arr)
    deposited_ion_vec_linalg  = np.linalg.solve(np.identity(eleceng.size) - sec_elec_spec_N_arr, deposited_ion_eng_arr)
    deposited_heat_vec_linalg = np.linalg.solve(np.identity(eleceng.size) - sec_elec_spec_N_arr, deposited_heat_eng_arr)
    
    continuum_engloss_vec_linalg = np.linalg.solve(np.identity(eleceng.size) - sec_elec_spec_N_arr, continuum_engloss_arr)
    
    sec_phot_tf_linalg = np.linalg.solve(np.identity(eleceng.size) - sec_elec_spec_N_arr, sec_phot_spec_N_arr)
    
    # Prompt: low energy e produced in secondary spectrum upon scattering (sec_lowengelec_N_arr).
    # T : high energy e produced (sec_highengelec_N_arr). 
    sec_lowengelec_tf_linalg = np.linalg.solve(np.identity(eleceng.size) - sec_highengelec_N_arr, sec_lowengelec_N_arr)
    # High energy electron loop to get fully resolved spectrum.
    for i, eng in zip(eleceng_high_ind, eleceng_high):
        
        phot_ICS_N = phot_ICS_tf.grid_vals[i]
        
        elec_ICS_N      = elec_ICS_tf.grid_vals[i]
        
        elec_exc_HI_N   = elec_exc_HI_tf.grid_vals[i]
        elec_exc_HeI_N  = elec_exc_HeI_tf.grid_vals[i]
        elec_exc_HeII_N = elec_exc_HeII_tf.grid_vals[i]
        
        elec_ion_HI_N   = elec_ion_HI_tf.grid_vals[i]
        elec_ion_HeI_N  = elec_ion_HeI_tf.grid_vals[i]
        elec_ion_HeII_N = elec_ion_HeII_tf.grid_vals[i]
                
        # elec_heat_spec = spectools.rebin_N_arr(np.array([1]), np.array([eng]), eleceng)
        # elec_heat_spec.eng -= dE_heat_dt[i]
        # elec_heat_spec.rebin(eleceng)
        # elec_heat_N = elec_heat_spec.N

        elec_heat_N = elec_heat_spec_grid[i]        
        
        sec_elec_spec_N = sec_elec_spec_N_arr[i]
        sec_phot_spec_N = phot_ICS_N
        

        sec_elec_totN = sec_elec_totN_arr[i]
        # The *net* total energy of secondary electrons produced
        # per unit time.
        sec_elec_toteng = sec_elec_toteng_arr[i]
        # The total energy of secondary photons produced per unit time.
        sec_phot_toteng = sec_phot_toteng_arr[i]
        # Deposited ICS energy per unit time, dD/dt.
        # Numerical error (should be zero except for numerics)
        deposited_ICS_eng = deposited_ICS_eng_arr[i]
        # Deposited excitation energy. 
        deposited_exc_eng = deposited_exc_eng_arr[i]
        # Deposited ionization energy. Remember that the secondary spectrum
        # has two electrons for each ionization event.
        deposited_ion_eng = deposited_ion_eng_arr[i]
        # Deposited heating energy. 
        deposited_heat_eng = deposited_heat_eng_arr[i]
        
        # In the original code, the energy of the electron has gamma > 20,
        # then the continuum energy loss is assigned 
        # to deposited_eng instead.
        # I'm not sure if this is necessary, but let's be consistent with the
        # original code for now.

#         continuum_engloss = CMB_upscatter_eng_rate
        
#         if eng + phys.me > 20*phys.me:
#             deposited_ICS_eng -= CMB_upscatter_eng_rate
#             continuum_engloss = 0

        continuum_engloss = continuum_engloss_arr[i]

        # Remove self-scattering.
#         sec_elec_spec_N[i] = 0
        
        # Rescale.
#         toteng_no_self_scatter = (
#             np.dot(sec_elec_spec_N, eleceng)
#             + np.dot(sec_phot_spec_N, photeng)
#             - continuum_engloss
#             + deposited_ICS_eng
#             + deposited_exc_eng
#             + deposited_ion_eng
#             + deposited_heat_eng
#         )
        toteng_no_self_scatter = toteng_no_self_scatter_arr[i]
        
#         fac = eng/toteng_no_self_scatter
        # Normalize to one electron. 
        
#         sec_elec_spec_N    *= fac
#         sec_phot_spec_N    *= fac
#         continuum_engloss  *= fac
#         deposited_ICS_eng  *= fac
#         deposited_exc_eng  *= fac
#         deposited_ion_eng  *= fac
#         deposited_heat_eng *= fac

        # Get the full secondary photon spectrum. Type 'N'
        resolved_phot_spec_vals = np.dot(
            sec_elec_spec_N, sec_phot_tf.grid_vals
        )
        
        # Get the full secondary low energy electron spectrum. Type 'N'.

        resolved_lowengelec_spec_vals = np.dot(
            sec_elec_spec_N, sec_lowengelec_tf.grid_vals
        )
        
        # Add the resolved spectrum to the first scatter.
        sec_phot_spec_N += resolved_phot_spec_vals

        # Resolve the secondary electron continuum loss and deposition.
        continuum_engloss += np.dot(sec_elec_spec_N, cont_loss_ICS_vec)

        deposited_ICS_eng  += np.dot(sec_elec_spec_N, deposited_ICS_vec)
        deposited_exc_eng  += np.dot(sec_elec_spec_N, deposited_exc_vec)
        deposited_ion_eng  += np.dot(sec_elec_spec_N, deposited_ion_vec)
        deposited_heat_eng += np.dot(sec_elec_spec_N, deposited_heat_vec)

        # Now, append the resulting spectrum to the transfer function.
        # Do this without calling append of course: just add to the zeros
        # that fill the current row in _grid_vals.
        sec_phot_tf._grid_vals[i] += sec_phot_spec_N
        sec_lowengelec_tf._grid_vals[i] += resolved_lowengelec_spec_vals
        # Set the correct values in cont_loss_vec and deposited_vec.
        
        cont_loss_ICS_vec[i]  = continuum_engloss
        deposited_ICS_vec[i]  = deposited_ICS_eng
        deposited_exc_vec[i]  = deposited_exc_eng
        deposited_ion_vec[i]  = deposited_ion_eng
        deposited_heat_vec[i] = deposited_heat_eng

        
        failed_conservation_check = False
        
        if check_conservation_eng:

            conservation_check = (
                eng
                - np.dot(sec_lowengelec_tf.grid_vals[i], eleceng)
                + cont_loss_ICS_vec[i]
                - np.dot(sec_phot_tf.grid_vals[i], photeng)
                - deposited_exc_vec[i]
                - deposited_ion_vec[i]
                - deposited_heat_vec[i]
            )

            if np.abs(conservation_check/eng) > 0.1:
                failed_conservation_check = True
                print('failed!')
                
            if verbose or failed_conservation_check:
                
                print('***************************************************')
                print('rs: ', rs)
                print('injected energy: ', eng)
                print(
                    'Energy in low energy electrons: ',
                    np.dot(sec_lowengelec_tf.grid_vals[i], eleceng)
                )
                print(
                    'Energy in low energy electrons (linalg): ',
                    np.dot(sec_lowengelec_tf_linalg[i], eleceng)
                )
                utils.compare_arr([eleceng, sec_lowengelec_tf.grid_vals[i], sec_lowengelec_tf_linalg[i]])
                print('Energy in photons: ', np.dot(sec_phot_tf.grid_vals[i], photeng))
                print('Energy in photons (linalg): ', np.dot(sec_phot_tf_linalg[i], photeng))
                print('Continuum_engloss: ', cont_loss_ICS_vec[i])
                print('Continuum_engloss (linalg): ', continuum_engloss_vec_linalg[i])
                
                print(
                    'Energy in photons - Continuum: ',
                    np.dot(sec_phot_tf.grid_vals[i], photeng) - cont_loss_ICS_vec[i]
                )
                print(
                    'Deposited in ionization: ', deposited_ion_vec[i]
                )
                print(
                    'Deposited in ionization (linalg): ', deposited_ion_vec_linalg[i]
                )
                print(
                    'Deposited in excitation: ', deposited_exc_vec[i]
                )
                print(
                    'Deposited in excitation (linalg): ', deposited_exc_vec_linalg[i]
                )
                print(
                    'Deposited in heating: ', deposited_heat_vec[i]
                )
                print(
                    'Deposited in heating (linalg): ', deposited_heat_vec_linalg[i]
                )
                print(
                    'Energy is conserved up to (%): ',
                    conservation_check/eng*100
                )
                print('Deposited in ICS: ', deposited_ICS_vec[i])
                print('Deposited in ICS (linalg): ', deposited_ICS_vec_linalg[i])
                print(
                    'Energy conservation with deposited (%): ',
                    (conservation_check - deposited_ICS_vec[i])/eng*100
                )
                print('***************************************************')
                
            if failed_conservation_check:
                raise RuntimeError('Conservation of energy failed.')


    return (
        sec_phot_tf, sec_lowengelec_tf,
        deposited_ion_vec, deposited_exc_vec, deposited_heat_vec,
        cont_loss_ICS_vec, deposited_ICS_vec
    )

In [125]:
%lprun -f get_elec_cooling_tf_fast_linalg get_elec_cooling_tf_fast_linalg(ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf, coll_ion_sec_elec_specs, coll_exc_sec_elec_specs, eleceng, photeng, rs, xe, xHe=0, ics_engloss_data = ics_engloss_data, check_conservation_eng=True, verbose=False)

In [154]:
%autoreload

from darkhistory.electrons.elec_cooling import get_elec_cooling_tf_fast_linalg

%lprun -f get_elec_cooling_tf_fast get_elec_cooling_tf_fast(ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf, coll_ion_sec_elec_specs, coll_exc_sec_elec_specs, eleceng, photeng, rs, xe, xHe=0, linalg=False, ics_engloss_data = ics_engloss_data, check_conservation_eng=True, verbose=False)




In [164]:
%%prun
for i in np.arange(120):
    get_elec_cooling_tf_fast(
        ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf, 
        coll_ion_sec_elec_specs, coll_exc_sec_elec_specs, 
        eleceng, photeng, rs, xe, xHe=0, 
        linalg=False, ics_engloss_data = ics_engloss_data, 
        check_conservation_eng=False, verbose=False
    )
    get_elec_cooling_tf_fast_linalg(
        ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf, 
        coll_ion_sec_elec_specs, coll_exc_sec_elec_specs, 
        eleceng, photeng, rs, xe, xHe=0, 
        ics_engloss_data = ics_engloss_data, 
        check_conservation_eng=False, verbose=False
    )